# Seznam - ...

...

Summary:

- Prediction type: __Regression model__
- Domain: __Sales__
- Prediction target: __Salaries__ 
- Population size: __23111__

_Author: Dr. Patrick Urbanke_

# Background

...

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/VOC) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.engine.set_project('seznam')



Connected to project 'seznam'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="Seznam",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='Seznam',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
dobito = load_if_needed("dobito")
probehnuto = load_if_needed("probehnuto")
probehnuto_mimo_penezenku = load_if_needed("probehnuto_mimo_penezenku")

In [5]:
dobito

name,client_id,month_year_datum_transakce,sluzba,kc_dobito
role,unused_float,unused_string,unused_string,unused_string
0,7157857,2012-10-01,c,1045.62
1,109700,2015-10-01,c,5187.28
2,51508,2015-08-01,c,408.20
3,9573550,2012-10-01,c,521.24
4,9774621,2014-11-01,c,386.22
,...,...,...,...
554341,65283,2012-09-01,g,7850.00
554342,6091446,2012-08-01,g,31400.00
554343,1264806,2013-08-01,g,-8220.52


In [6]:
probehnuto

name,client_id,month_year_datum_transakce,sluzba,kc_proklikano
role,unused_float,unused_string,unused_string,unused_string
0,109145,2013-06-01,c,-31.40
1,9804394,2015-10-01,h,37.68
2,9803353,2015-10-01,h,725.34
3,9801753,2015-10-01,h,194.68
4,9800425,2015-10-01,h,1042.48
,...,...,...,...
1462073,98857,2015-08-01,NULL,153.86
1462074,95776,2015-09-01,NULL,153.86
1462075,98857,2015-09-01,NULL,153.86


In [7]:
probehnuto_mimo_penezenku

name,client_id,Month/Year,probehla_inzerce_mimo_penezenku
role,unused_float,unused_string,unused_string
0,3901,2012-08-01,ANO
1,3901,2012-09-01,ANO
2,3901,2012-10-01,ANO
3,3901,2012-11-01,ANO
4,3901,2012-12-01,ANO
,...,...,...
599381,9804086,2015-10-01,ANO
599382,9804238,2015-10-01,ANO
599383,9804782,2015-10-01,ANO


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
dobito.set_role("client_id", getml.data.roles.join_key)
dobito.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
dobito.set_role("sluzba", getml.data.roles.categorical)
dobito.set_role("kc_dobito", getml.data.roles.numerical)

dobito.set_unit("sluzba", "service")

dobito

name,month_year_datum_transakce,client_id,sluzba,kc_dobito
role,time_stamp,join_key,categorical,numerical
unit,"time stamp, comparison only",,service,
0,2012-10-01,7157857,c,1045.62
1,2015-10-01,109700,c,5187.28
2,2015-08-01,51508,c,408.2
3,2012-10-01,9573550,c,521.24
4,2014-11-01,9774621,c,386.22
,...,...,...,...
554341,2012-09-01,65283,g,7850
554342,2012-08-01,6091446,g,31400


In [9]:
probehnuto.set_role("client_id", getml.data.roles.join_key)
probehnuto.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
probehnuto.set_role("sluzba", getml.data.roles.categorical)
probehnuto.set_role("kc_proklikano", getml.data.roles.target)

probehnuto.set_unit("sluzba", "service")

probehnuto

name,month_year_datum_transakce,client_id,kc_proklikano,sluzba
role,time_stamp,join_key,target,categorical
unit,"time stamp, comparison only",,,service
0,2013-06-01,109145,-31.4,c
1,2015-10-01,9804394,37.68,h
2,2015-10-01,9803353,725.34,h
3,2015-10-01,9801753,194.68,h
4,2015-10-01,9800425,1042.48,h
,...,...,...,...
1462073,2015-08-01,98857,153.86,NULL
1462074,2015-09-01,95776,153.86,NULL


In [10]:
probehnuto_mimo_penezenku.set_role("client_id", getml.data.roles.join_key)
probehnuto_mimo_penezenku.set_role("Month/Year", getml.data.roles.time_stamp)

probehnuto_mimo_penezenku

name,Month/Year,client_id,probehla_inzerce_mimo_penezenku
role,time_stamp,join_key,unused_string
unit,"time stamp, comparison only",,
0,2012-08-01,3901,ANO
1,2012-09-01,3901,ANO
2,2012-10-01,3901,ANO
3,2012-11-01,3901,ANO
4,2012-12-01,3901,ANO
,...,...,...
599381,2015-10-01,9804086,ANO
599382,2015-10-01,9804238,ANO


In [11]:
split = getml.data.split.random(train=0.8, test=0.2)
split

,
0,train
1,train
2,train
3,test
4,train
,...


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [12]:
star_schema = getml.data.StarSchema(population=probehnuto, alias="population", split=split)

star_schema.join(
    probehnuto,
    on="client_id",
    time_stamps="month_year_datum_transakce",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema.join(
    dobito,
    on="client_id",
    time_stamps="month_year_datum_transakce",
)

star_schema.join(
    probehnuto_mimo_penezenku,
    on="client_id", 
    time_stamps=("month_year_datum_transakce",  "Month/Year"),
)

star_schema

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [13]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,    
    sampling_factor=0.1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [14]:
pipe1 = getml.pipeline.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

### 2.3 Model training

In [15]:
pipe1.check(star_schema.train)

Checking data model...


Preprocessing...
[========================================] 100%

INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


In [16]:
pipe1.fit(star_schema.train)

Checking data model...


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Preprocessing...
[========================================] 100%

FastProp: Try

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-iIW3O5'])

url: http://localhost:1709/#/getpipeline/seznam/ZPMoHO/0/

### 2.4 Model evaluation

In [17]:
pipe1.score(star_schema.test)



Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-07-19 16:41:09,train,kc_proklikano,3090.3283,22460.03,0.8579
1,2021-07-19 16:43:12,test,kc_proklikano,3160.0238,24479.1592,0.7822


### 2.5 featuretools

In [18]:
include = (getml.data.random() < 0.25)
include

,
0,true
1,false
2,true
3,false
4,false
,...


In [19]:
population_train_pd = star_schema.train.population[include].to_pandas()
population_test_pd = star_schema.test.population.to_pandas()

In [20]:
population_train_pd["id"] = population_train_pd.index
population_test_pd["id"] = population_test_pd.index

In [21]:
probehnuto_pd = probehnuto.drop(probehnuto.unused_names).to_pandas()
dobito_pd = dobito.drop(dobito.unused_names).to_pandas()
probehnuto_mimo_penezenku_pd = probehnuto_mimo_penezenku.drop(probehnuto_mimo_penezenku.unused_names).to_pandas()

In [22]:
def prepare_peripheral(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["month_year_datum_transakce_x"] < peripheral_new["month_year_datum_transakce_y"]
    ]

    del peripheral_new["month_year_datum_transakce_y"]
    del peripheral_new["client_id"]

    return peripheral_new.rename({"month_year_datum_transakce_y": "month_year_datum_transakce"})

In [23]:
def prepare_probehnuto_mimo_penezenku(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["Month/Year"] < peripheral_new["month_year_datum_transakce"]
    ]

    del peripheral_new["month_year_datum_transakce"]
    del peripheral_new["client_id"]

    return peripheral_new

In [24]:
dobito_train_pd = prepare_peripheral(dobito_pd, population_train_pd)
dobito_test_pd = prepare_peripheral(dobito_pd, population_test_pd)
dobito_train_pd

,month_year_datum_transakce_x,sluzba,kc_dobito,id
0,2012-10-01,c,1045.62,2127
1,2012-10-01,c,1045.62,17709
2,2012-10-01,c,1045.62,50363
3,2015-03-01,a,25214.20,2127
4,2015-03-01,a,25214.20,17709
...,...,...,...,...
4462025,2012-08-01,g,37680.00,217660
4462026,2012-08-01,g,37680.00,235321
4462027,2012-08-01,g,37680.00,253661
4462028,2012-08-01,g,37680.00,264765


In [25]:
probehnuto_train_pd = prepare_peripheral(probehnuto_pd, population_train_pd)
probehnuto_test_pd = prepare_peripheral(probehnuto_pd, population_test_pd)
probehnuto_train_pd

,month_year_datum_transakce_x,kc_proklikano,sluzba,id
1,2013-06-01,-31.40,c,281262
4,2013-06-01,-31.40,c,288356
6,2013-06-01,-31.40,c,289265
7,2013-06-01,-31.40,c,289267
10,2013-06-01,-31.40,c,290759
...,...,...,...,...
11186670,2012-08-01,948.28,NULL,291919
11186671,2012-09-01,979.68,e,291919
11186673,2012-08-01,948.28,NULL,291848
11186674,2012-08-01,87.92,e,291848


In [26]:
probehnuto_mimo_penezenku_train_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_train_pd)
probehnuto_mimo_penezenku_test_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_test_pd)
probehnuto_mimo_penezenku_train_pd

,Month/Year,id
0,2012-08-01,269301
8,2012-08-01,9204
9,2012-08-01,23838
10,2012-08-01,24471
11,2012-08-01,45868
...,...,...
3568048,2015-09-01,160015
3568050,2015-09-01,19
3568051,2015-09-01,1565
3568053,2015-09-01,151283


In [27]:
del population_train_pd["client_id"]
del population_test_pd["client_id"]

In [28]:
population_train_pd

,month_year_datum_transakce,kc_proklikano,sluzba,id
0,2013-06-01,-31.40,c,0
1,2015-10-01,725.34,h,1
2,2015-10-01,8550.22,h,2
3,2015-10-01,2408.38,h,3
4,2015-10-01,1893.42,h,4
...,...,...,...,...
292153,2015-03-01,153.86,NULL,292153
292154,2015-05-01,153.86,NULL,292154
292155,2015-06-01,13545.96,NULL,292155
292156,2015-06-01,153.86,NULL,292156


In [29]:
entities_train = {
    "population" : (population_train_pd, "id"),
    "dobito": (dobito_train_pd, "index"),
    "probehnuto": (probehnuto_train_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_train_pd, "index"),
}

In [30]:
entities_test = {
    "population" : (population_test_pd, "id"),
    "dobito": (dobito_test_pd, "index"),
    "probehnuto": (probehnuto_test_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_test_pd, "index"),
}

In [31]:
relationships = [
    ("population", "id", "dobito", "id"),
    ("population", "id", "probehnuto", "id"),
    ("population", "id", "probehnuto_mimo_penezenku", "id"),
]

In [32]:
featuretools_train_pd = featuretools.dfs(
    entities=entities_train,
    relationships=relationships,
    target_entity="population")[0]

/home/patrick/.local/lib/python3.9/site-packages/featuretools/entityset/entity.py:462: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn("index {} not found in dataframe, creating new "


In [33]:
featuretools_test_pd = featuretools.dfs(
    entities=entities_test,
    relationships=relationships,
    target_entity="population")[0]

In [34]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [35]:
featuretools_train.set_role("kc_proklikano", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.unused_float_names, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.unused_string_names, getml.data.roles.categorical)

featuretools_train

name,kc_proklikano,sluzba,MODE(dobito.sluzba),MODE(probehnuto.sluzba),COUNT(dobito),MAX(dobito.kc_dobito),MEAN(dobito.kc_dobito),MIN(dobito.kc_dobito),NUM_UNIQUE(dobito.sluzba),SKEW(dobito.kc_dobito),STD(dobito.kc_dobito),SUM(dobito.kc_dobito),COUNT(probehnuto),MAX(probehnuto.kc_proklikano),MEAN(probehnuto.kc_proklikano),MIN(probehnuto.kc_proklikano),NUM_UNIQUE(probehnuto.sluzba),SKEW(probehnuto.kc_proklikano),STD(probehnuto.kc_proklikano),SUM(probehnuto.kc_proklikano),COUNT(probehnuto_mimo_penezenku),DAY(month_year_datum_transakce),MONTH(month_year_datum_transakce),WEEKDAY(month_year_datum_transakce),YEAR(month_year_datum_transakce),MODE(dobito.DAY(month_year_datum_transakce_x)),MODE(dobito.MONTH(month_year_datum_transakce_x)),MODE(dobito.WEEKDAY(month_year_datum_transakce_x)),MODE(dobito.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.YEAR(month_year_datum_transakce_x)),MODE(probehnuto.DAY(month_year_datum_transakce_x)),MODE(probehnuto.MONTH(month_year_datum_transakce_x)),MODE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),MODE(probehnuto.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.YEAR(month_year_datum_transakce_x)),MODE(probehnuto_mimo_penezenku.DAY(Month/Year)),MODE(probehnuto_mimo_penezenku.MONTH(Month/Year)),MODE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),MODE(probehnuto_mimo_penezenku.YEAR(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.DAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.MONTH(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.YEAR(Month/Year))
role,target,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical
0,-31.4,c,c,NULL,1,1306.24,1306.24,1306.24,1,nan,nan,1306.24,13,351.68,155.7923,9.42,2,0.5817,79.3799,2025.3,0,1,6,5,2013,1,12,5,2012,1,1,1,1,1,8,0,2012,1,10,6,2,nan,nan,nan,nan,nan,nan,nan,nan
1,725.34,h,h,h,4,1036.2,614.655,257.48,1,0.5563,324.3624,2458.62,5,634.28,388.732,131.88,1,0.09478,205.8605,1943.66,0,1,10,3,2015,1,5,0,2015,1,4,4,1,1,5,0,2015,1,5,5,1,nan,nan,nan,nan,nan,nan,nan,nan
2,8550.22,h,h,h,7,62800,20907.9143,0,2,1.214,22375.0788,146355.4,11,39752.4,13280.7727,3215.36,1,1.4845,12240.1205,146088.5,0,1,10,3,2015,1,8,2,2015,1,6,5,2,1,1,2,2015,1,11,7,2,nan,nan,nan,nan,nan,nan,nan,nan
3,2408.38,h,h,h,4,1554.3,1361.19,1296.82,1,2,128.74,5444.76,5,1635.94,1092.72,15.7,1,-1.0846,711.3296,5463.6,0,1,10,3,2015,1,5,1,2015,1,4,4,1,1,5,0,2015,1,5,5,1,nan,nan,nan,nan,nan,nan,nan,nan
4,1893.42,h,h,d,12,2615.62,1078.3283,310.86,4,0.8429,835.3628,12939.94,22,1510.34,471.,0,4,0.8615,531.1884,10362.,0,1,10,3,2015,1,2,4,2015,1,5,6,2,1,2,2,2015,1,9,6,2,nan,nan,nan,nan,nan,nan,nan,nan
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292153,153.86,NULL,c,NULL,12,518.1,337.8117,153.86,1,0.02976,99.6511,4053.74,34,188.4,141.9465,0,2,-2.9319,44.2498,4826.18,0,1,3,6,2015,1,1,0,2013,1,8,6,3,1,1,5,2013,1,12,7,4,nan,nan,nan,nan,nan,nan,nan,nan
292154,153.86,NULL,f,NULL,6,1296.82,913.74,31.4,1,-2.1286,444.4448,5482.44,34,188.4,150.8124,-31.4,2,-5.2155,33.226,5127.62,0,1,5,4,2015,1,2,1,2013,1,6,4,3,1,1,5,2013,1,12,7,4,nan,

In [36]:
featuretools_test.set_role("kc_proklikano", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.unused_float_names, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.unused_string_names, getml.data.roles.categorical)

featuretools_test

name,kc_proklikano,sluzba,MODE(dobito.sluzba),MODE(probehnuto.sluzba),COUNT(dobito),MAX(dobito.kc_dobito),MEAN(dobito.kc_dobito),MIN(dobito.kc_dobito),NUM_UNIQUE(dobito.sluzba),SKEW(dobito.kc_dobito),STD(dobito.kc_dobito),SUM(dobito.kc_dobito),COUNT(probehnuto),MAX(probehnuto.kc_proklikano),MEAN(probehnuto.kc_proklikano),MIN(probehnuto.kc_proklikano),NUM_UNIQUE(probehnuto.sluzba),SKEW(probehnuto.kc_proklikano),STD(probehnuto.kc_proklikano),SUM(probehnuto.kc_proklikano),COUNT(probehnuto_mimo_penezenku),DAY(month_year_datum_transakce),MONTH(month_year_datum_transakce),WEEKDAY(month_year_datum_transakce),YEAR(month_year_datum_transakce),MODE(dobito.DAY(month_year_datum_transakce_x)),MODE(dobito.MONTH(month_year_datum_transakce_x)),MODE(dobito.WEEKDAY(month_year_datum_transakce_x)),MODE(dobito.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.YEAR(month_year_datum_transakce_x)),MODE(probehnuto.DAY(month_year_datum_transakce_x)),MODE(probehnuto.MONTH(month_year_datum_transakce_x)),MODE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),MODE(probehnuto.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.YEAR(month_year_datum_transakce_x)),MODE(probehnuto_mimo_penezenku.DAY(Month/Year)),MODE(probehnuto_mimo_penezenku.MONTH(Month/Year)),MODE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),MODE(probehnuto_mimo_penezenku.YEAR(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.DAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.MONTH(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.YEAR(Month/Year))
role,target,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical
0,194.68,h,d,d,2,1296.82,777.15,257.48,2,nan,734.9244,1554.3,2,763.02,401.92,40.82,2,nan,510.6725,803.84,0,1,10,3,2015,1,9,1,2015,1,1,1,1,1,9,1,2015,1,1,1,1,nan,nan,nan,nan,nan,nan,nan,nan
1,405.06,h,h,h,1,1296.82,1296.82,1296.82,1,nan,nan,1296.82,2,565.2,452.16,339.12,1,nan,159.8627,904.32,0,1,10,3,2015,1,8,5,2015,1,1,1,1,1,8,1,2015,1,2,2,1,nan,nan,nan,nan,nan,nan,nan,nan
2,580.9,h,d,d,4,1296.82,1231.665,1036.2,2,-2.,130.31,4926.66,5,913.74,454.044,34.54,2,0.2893,328.7162,2270.22,0,1,10,3,2015,1,9,1,2015,1,3,3,1,1,9,1,2015,1,4,4,1,nan,nan,nan,nan,nan,nan,nan,nan
3,106.76,h,NULL,NULL,0,nan,nan,nan,nan,nan,nan,0,0,nan,nan,nan,nan,nan,nan,0,0,1,10,3,2015,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1927.96,h,d,d,15,7784.06,1850.088,257.48,2,2.4789,1898.9207,27751.32,21,5199.84,1148.1933,25.12,2,1.8651,1342.4638,24112.06,0,1,10,3,2015,1,9,0,2015,1,10,6,2,1,9,0,2015,1,12,7,2,nan,nan,nan,nan,nan,nan,nan,nan
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292828,153.86,NULL,c,NULL,5,1306.24,1045.62,31.4,1,-2.2358,566.9809,5228.1,36,351.68,150.3711,-31.4,3,-0.2998,56.2491,5413.36,0,1,4,2,2015,1,12,5,2013,1,4,4,3,1,8,5,2013,1,12,7,4,nan,nan,nan,nan,nan,nan,nan,nan
292829,153.86,NULL,c,NULL,3,2615.62,1757.3533,62.8,1,-1.7316,1467.5674,5272.06,35,188.4,150.0023,-62.8,2,-5.4539,37.9032,5250.08,0,1,6,0,2015,1,4,1,2012,1,3,3,3,1,1,5,2013,1,12,7,4,nan,nan,nan,nan,nan,

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [37]:
data_model = getml.data.DataModel("population")

In [38]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.pipeline.Pipeline(
    tags=['featuretools'],
    data_model=data_model,
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [39]:
pipe2.fit(featuretools_train)

Checking data model...


Preprocessing...
[========================================] 100%

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'COALESCE( day(month_year_datum_transakce), 1.000000 )' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'COALESCE( mode(dobito.day(month_year_datum_transakce_x)), 1.000000 )' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'COALESCE( num_unique(dobito.day(month_year_datum_transakce_x)), 1.000000 )' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

url: http://localhost:1709/#/getpipeline/seznam/Hw09qo/0/

In [40]:
pipe2.score(featuretools_test)



Preprocessing...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-07-19 17:25:48,featuretools_train,kc_proklikano,5035.4814,23178.6853,0.8417
1,2021-07-19 17:25:50,featuretools_test,kc_proklikano,5167.4352,31655.1492,0.6324


### 2.7 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared  | RMSE   | MAE
-------------------- | ---------- | -------| ----
getML: FastProp      |     78.22% | 24,480 | 3,160
featuretools         |     63.24% | 31,655 | 5,167

## 3. Conclusion

...

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.